<a href="https://colab.research.google.com/github/andrewdk1123/KoSentiment/blob/main/LSTMSentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [24]:
# Load packages
import pandas as pd
import io
import re
from transformers import BertTokenizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random

In [1]:
from google.colab import files

uploaded = files.upload()

Saving training.csv to training.csv


In [2]:
print(uploaded.keys())

dict_keys(['training.csv'])


In [8]:
train_data = pd.read_csv(io.BytesIO(uploaded['training.csv']), sep = '\t')
train_data.head()

,emotion,sentence
0,E18,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,E18,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,E18,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,E18,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,E18,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.


In [9]:
uploaded = files.upload()

Saving test.csv to test.csv


In [10]:
print(uploaded.keys())

dict_keys(['test.csv'])


In [11]:
test_data = pd.read_csv(io.BytesIO(uploaded['test.csv']), sep = '\t')
test_data.head()

,emotion,sentence
0,E31,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...
1,E31,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.
2,E31,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.
3,E31,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.
4,E31,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.


In [12]:
# Convert labels to 'pos' and 'neg'
train_data['label'] = train_data['emotion'].apply(lambda x: 'pos' if x in ['E60', 'E61', 'E62', 'E63', 'E64', 'E65', 'E66', 'E67', 'E68', 'E69'] else 'neg')
test_data['label'] = test_data['emotion'].apply(lambda x: 'pos' if x in ['E60', 'E61', 'E62', 'E63', 'E64', 'E65', 'E66', 'E67', 'E68', 'E69'] else 'neg')

# Convert categorical labels to binary
train_data['label'] = train_data['label'].map({'pos': 1, 'neg': 0})
test_data['label'] = test_data['label'].map({'pos': 1, 'neg': 0})

train_data.iloc[:10,:]

,emotion,sentence,label
0,E18,일은 왜 해도 해도 끝이 없을까? 화가 난다.,0
1,E18,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,0
2,E18,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,0
3,E18,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,0
4,E18,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,0
5,E18,직장에 다니고 있지만 시간만 버리는 거 같아. 진지하게 진로에 대한 고민이 생겨.,0
6,E18,성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔. 나도 섭섭해.,0
7,E66,퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.,1
8,E37,졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어.,0
9,E66,요즘 직장생활이 너무 편하고 좋은 것 같아!,1


In [15]:
# Load and apply KoBERT tokenizer to sentence
tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')

train_data['tokenized_sentence'] = train_data['sentence'].apply(lambda x: tokenizer.tokenize(x))
test_data['tokenized_sentence'] = test_data['sentence'].apply(lambda x: tokenizer.tokenize(x))

train_data.iloc[:10,:]

tokenizer_config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

,emotion,sentence,label,tokenized_sentence
0,E18,일은 왜 해도 해도 끝이 없을까? 화가 난다.,0,"[일은, 왜, 해도, 해도, 끝이, 없을까, ?, 화가, 난다, .]"
1,E18,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,0,"[이번, 달에, 또, 급여, ##가, 깎, ##였어, !, 물가, ##는, 오르는,..."
2,E18,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,0,"[회사에, 신입, ##이, 들어왔, ##는데, 말투, ##가, 거슬, ##려, .,..."
3,E18,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,0,"[직장, ##에서, 막내, ##라는, 이유로, 나에게, ##만, 온갖, 심, ##부..."
4,E18,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,0,"[얼마, 전, 입사, ##한, 신입, ##사원, ##이, 나를, 무시, ##하는, ..."
5,E18,직장에 다니고 있지만 시간만 버리는 거 같아. 진지하게 진로에 대한 고민이 생겨.,0,"[직장, ##에, 다니고, 있지만, 시간, ##만, 버리는, 거, 같아, ., 진지..."
6,E18,성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔. 나도 섭섭해.,0,"[성인, ##인데도, 진로, ##를, 아직도, 못, 정, ##했다고, 부모님이, 노..."
7,E66,퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.,1,"[퇴, ##사한, 지, 얼마, 안, 됐, ##지만, 천천히, 직장, ##을, 구해,..."
8,E37,졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어.,0,"[졸업, ##반이, ##라서, 취업, ##을, 생각해, ##야, 하는데, 지금, 너..."
9,E66,요즘 직장생활이 너무 편하고 좋은 것 같아!,1,"[요즘, 직장, ##생활, ##이, 너무, 편하고, 좋은, 것, 같아, !]"


In [18]:
# Function to clean tokens
def clean_tokens(tokens):
    cleaned_tokens = []
    for token in tokens:
        if token.startswith('##'):  # remove subwords
            continue
        token = re.sub(r'[^\w\s]', '', token)  # remove punctuation
        if token:  # if token is not empty
            cleaned_tokens.append(token)
    return cleaned_tokens

# Apply clean_tokens to the tokenized_sentence columns in train_data and test_data
train_data['cleaned_tokens'] = train_data['tokenized_sentence'].apply(clean_tokens)
test_data['cleaned_tokens'] = test_data['tokenized_sentence'].apply(clean_tokens)

train_data.iloc[:10,:]

,emotion,sentence,label,tokenized_sentence,cleaned_tokens
0,E18,일은 왜 해도 해도 끝이 없을까? 화가 난다.,0,"[일은, 왜, 해도, 해도, 끝이, 없을까, ?, 화가, 난다, .]","[일은, 왜, 해도, 해도, 끝이, 없을까, 화가, 난다]"
1,E18,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,0,"[이번, 달에, 또, 급여, ##가, 깎, ##였어, !, 물가, ##는, 오르는,...","[이번, 달에, 또, 급여, 깎, 물가, 오르는, 월급, 자꾸, 깎, 너무, 화가, 나]"
2,E18,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,0,"[회사에, 신입, ##이, 들어왔, ##는데, 말투, ##가, 거슬, ##려, .,...","[회사에, 신입, 들어왔, 말투, 거슬, 그런, 애를, 매일, 봐야, 한다고, 생각..."
3,E18,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,0,"[직장, ##에서, 막내, ##라는, 이유로, 나에게, ##만, 온갖, 심, ##부...","[직장, 막내, 이유로, 나에게, 온갖, 심, 시켜, 일도, 많은, 데, 정말, 분..."
4,E18,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,0,"[얼마, 전, 입사, ##한, 신입, ##사원, ##이, 나를, 무시, ##하는, ...","[얼마, 전, 입사, 신입, 나를, 무시, 것, 같아서, 너무, 화가, 나]"
5,E18,직장에 다니고 있지만 시간만 버리는 거 같아. 진지하게 진로에 대한 고민이 생겨.,0,"[직장, ##에, 다니고, 있지만, 시간, ##만, 버리는, 거, 같아, ., 진지...","[직장, 다니고, 있지만, 시간, 버리는, 거, 같아, 진지하게, 진로, 대한, 고..."
6,E18,성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔. 나도 섭섭해.,0,"[성인, ##인데도, 진로, ##를, 아직도, 못, 정, ##했다고, 부모님이, 노...","[성인, 진로, 아직도, 못, 정, 부모님이, 노, 나도, 섭섭]"
7,E66,퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.,1,"[퇴, ##사한, 지, 얼마, 안, 됐, ##지만, 천천히, 직장, ##을, 구해,...","[퇴, 지, 얼마, 안, 됐, 천천히, 직장, 구해]"
8,E37,졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어.,0,"[졸업, ##반이, ##라서, 취업, ##을, 생각해, ##야, 하는데, 지금, 너...","[졸업, 취업, 생각해, 하는데, 지금, 너무, 느, 이래, 되나, 싶어]"
9,E66,요즘 직장생활이 너무 편하고 좋은 것 같아!,1,"[요즘, 직장, ##생활, ##이, 너무, 편하고, 좋은, 것, 같아, !]","[요즘, 직장, 너무, 편하고, 좋은, 것, 같아]"


## Undersample

In [45]:
import torch
from torch.nn.utils.rnn import pad_sequence

# Build a vocabulary mapping
vocab = {word: idx + 1 for idx, (word, _) in enumerate(tokenizer.get_vocab().items())}

# Create reverse_vocab by swapping keys and values
reverse_vocab = {idx: word for word, idx in vocab.items()}

# Convert cleaned tokens to numerical indices
train_data['numerical_tokens'] = train_data['cleaned_tokens'].apply(lambda tokens: [vocab.get(token, 0) for token in tokens])

# Pad sequences using PyTorch
padded_sequences = pad_sequence([torch.tensor(tokens) for tokens in train_data['numerical_tokens']], batch_first=True, padding_value=0)

# Add the padded_sequences to your DataFrame
train_data['padded_sequences'] = [list(seq) for seq in padded_sequences]

# Display the DataFrame
print(train_data['padded_sequences'])

0        [tensor(17193), tensor(5731), tensor(14709), t...
1        [tensor(14100), tensor(27505), tensor(3548), t...
2        [tensor(25384), tensor(25952), tensor(28232), ...
3        [tensor(17808), tensor(25722), tensor(17655), ...
4        [tensor(14333), tensor(6020), tensor(33165), t...
                               ...                        
51623    [tensor(19836), tensor(14304), tensor(14131), ...
51624    [tensor(17091), tensor(14012), tensor(5556), t...
51625    [tensor(14131), tensor(14441), tensor(14450), ...
51626    [tensor(4146), tensor(2899), tensor(14058), te...
51627    [tensor(27579), tensor(15389), tensor(6267), t...
Name: padded_sequences, Length: 51628, dtype: object


In [20]:
print("Max Sequence Length:", max(len(tokens) for tokens in train_data['cleaned_tokens']))
print(len(train_data['padded_sequences'][0]) == max(len(tokens) for tokens in train_data['cleaned_tokens']))

print(train_data['padded_sequences'][0])

Max Sequence Length: 39
True
[tensor(17193), tensor(5731), tensor(14709), tensor(14709), tensor(19771), tensor(30323), tensor(21845), tensor(20939), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0)]


In [21]:
test_data['tokenized_sentence'] = test_data['sentence'].apply(lambda x: tokenizer.tokenize(x))


# Function to clean tokens
def clean_tokens(tokens):
    cleaned_tokens = []
    for token in tokens:
        if token.startswith('##'):  # remove subwords
            continue
        token = re.sub(r'[^\w\s]', '', token)  # remove punctuation
        if token:  # if token is not empty
            cleaned_tokens.append(token)
    return cleaned_tokens

test_data['cleaned_tokens'] = test_data['tokenized_sentence'].apply(clean_tokens)


In [22]:
# Convert cleaned tokens to numerical indices using the vocabulary mapping from training data
test_data['numerical_tokens'] = test_data['cleaned_tokens'].apply(lambda tokens: [vocab.get(token, 0) for token in tokens])

# Pad sequences using PyTorch
padded_sequences_test = pad_sequence([torch.tensor(tokens) for tokens in test_data['numerical_tokens']], batch_first=True, padding_value=0)

# Add the padded_sequences to your DataFrame
test_data['padded_sequences'] = [list(seq) for seq in padded_sequences_test]

In [25]:
# Define the LSTM model
class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, bidirectional, dropout):
        super(SentimentLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        lstm_output, _ = self.lstm(embedded)
        final_output = self.fc(lstm_output[:, -1, :])
        return final_output


# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, train_data['label'].values, test_size=0.2, random_state=1123)

# Convert data to PyTorch tensors and create DataLoader
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
val_dataset = torch.utils.data.TensorDataset(torch.tensor(X_val), torch.tensor(y_val))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define hyperparameters
vocab_size = len(vocab) + 1  # Additional 1 for the padding token
embedding_dim = 100
hidden_dim = 128
output_dim = 1  # Binary classification (positive or negative sentiment)
num_layers = 2
bidirectional = True
dropout = 0.5

# Instantiate the LSTM model
lstm_model = SentimentLSTM(vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, bidirectional, dropout)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lstm_model.to(device)

for epoch in range(num_epochs):
    lstm_model.train()
    for batch_data, batch_labels in train_loader:
        batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)

        # Forward pass
        outputs = lstm_model(batch_data)
        loss = criterion(outputs.squeeze(), batch_labels.float())

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    lstm_model.eval()
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        for val_batch_data, val_batch_labels in val_loader:
            val_batch_data, val_batch_labels = val_batch_data.to(device), val_batch_labels.to(device)

            val_outputs = lstm_model(val_batch_data)
            val_predictions.extend(torch.round(torch.sigmoid(val_outputs)).cpu().numpy())
            val_true_labels.extend(val_batch_labels.cpu().numpy())

    val_accuracy = accuracy_score(val_true_labels, val_predictions)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Validation Accuracy: {val_accuracy}")


<ipython-input-25-a7836882be76>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
<ipython-input-25-a7836882be76>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = torch.utils.data.TensorDataset(torch.tensor(X_val), torch.tensor(y_val))


Epoch 1/5, Loss: 0.2171744704246521, Validation Accuracy: 0.8781716056556266
Epoch 2/5, Loss: 0.40171849727630615, Validation Accuracy: 0.8781716056556266
Epoch 3/5, Loss: 0.49474674463272095, Validation Accuracy: 0.8781716056556266
Epoch 4/5, Loss: 0.2128443568944931, Validation Accuracy: 0.8781716056556266
Epoch 5/5, Loss: 0.5015712976455688, Validation Accuracy: 0.8781716056556266


In [26]:
# Create DataLoader for test data
test_dataset = torch.utils.data.TensorDataset(torch.tensor(padded_sequences_test), torch.tensor(test_data['label'].values))
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

lstm_model.eval()
test_predictions = []
test_true_labels = []

with torch.no_grad():
    for test_batch_data, test_batch_labels in test_loader:
        test_batch_data, test_batch_labels = test_batch_data.to(device), test_batch_labels.to(device)

        test_outputs = lstm_model(test_batch_data)
        test_predictions.extend(torch.round(torch.sigmoid(test_outputs)).cpu().numpy())
        test_true_labels.extend(test_batch_labels.cpu().numpy())

test_accuracy = accuracy_score(test_true_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy}")


Test Accuracy: 0.8173192771084338


<ipython-input-26-fcaa35ef06a3>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_dataset = torch.utils.data.TensorDataset(torch.tensor(padded_sequences_test), torch.tensor(test_data['label'].values))


In [27]:
# Select a random example from the test set
random_index = random.randint(0, len(test_data) - 1)
example_text = test_data.iloc[random_index]['sentence']
example_label = test_data.iloc[random_index]['label']

# Tokenize and preprocess the example
tokenized_example = tokenizer.tokenize(example_text)
cleaned_example = clean_tokens(tokenized_example)
numerical_example = [vocab.get(token, 0) for token in cleaned_example]

# Pad the sequence
padded_example = pad_sequence([torch.tensor(numerical_example)], batch_first=True, padding_value=0)

# Convert to PyTorch tensor and move to the appropriate device
example_tensor = padded_example.to(device)

# Set the model to evaluation mode
lstm_model.eval()

# Make a prediction
with torch.no_grad():
    output = lstm_model(example_tensor)
    predicted_label = torch.round(torch.sigmoid(output)).cpu().item()

# Display the example and prediction
print("Example Text:", example_text)
print("True Label:", example_label)
print("Predicted Label:", int(predicted_label))


Example Text: 사회 생활을 너무 일찍 시작해서 그런지 내 고민을 함께 나눌 사람이 없는 것 같아.
True Label: 0
Predicted Label: 0
